# We try to replace the (exact-match) cache with a smooth Cassandra

In [ ]:
from langchain.llms import OpenAI

In [ ]:
import langchain

In [ ]:
from CassandraCache import CassandraCache

In [ ]:
# This is where the session is created to later pass it to the CassandraCache
import os
from dotenv import load_dotenv
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

load_dotenv()

ASTRA_DB_SECURE_BUNDLE_PATH = os.environ["ASTRA_DB_SECURE_BUNDLE_PATH"]
ASTRA_DB_CLIENT_ID = "token"
ASTRA_DB_APPLICATION_TOKEN = os.environ["ASTRA_DB_APPLICATION_TOKEN"]

cluster = Cluster(
    cloud={
        "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
    },
    auth_provider=PlainTextAuthProvider(
        ASTRA_DB_CLIENT_ID,
        ASTRA_DB_APPLICATION_TOKEN,
    ),
)
astraSession = cluster.connect()

In [ ]:
langchain.llm_cache = CassandraCache(session=astraSession, keyspace='langchain')

In [ ]:
llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2)

In [ ]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm("How many eyes do spiders have?")

In [ ]:
%%time
# This time we expect a much shorter answer time
llm("How many eyes do spiders have?")

## You may want to check the Astra table now.